In [37]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("estate_project")

LangSmith 추적을 시작합니다.
[프로젝트명]
estate_project


In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [2]:
# 단계 1 : 문서 로드(Load Documents)
# !pip install pymupdf
#-----------PDF 문서 로드-----------
'''
loader = PyMuPDFLoader("SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")
'''

from langchain_community.document_loaders.csv_loader import CSVLoader
import pandas as pd
from pathlib import Path

'''
folder = Path("./ex_data/")
dfs = []
for file in folder.glob("*.csv"):
    df = pd.read_csv(file)
    df['source']=file.name
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

def row_to_text(row):
    return f"{row['region']} 지역의 {row['rletTpNm']}은 가격 {row['prc']}, 전세 {row['rentPrc']}, 면적 {row['spc1']} 평입니다. (출처: {row['source']})"

texts = df_all.apply(row_to_text, axis=1).tolist()

print(texts)
'''

# CSV 파일 로드
# loader = CSVLoader(file_path='../../estate_data/df_price_overall.csv', encoding='utf-8-sig')
loader = CSVLoader(file_path='./test_estate.csv', encoding='utf-8-sig')
docs = loader.load()
print(len(docs))
print(docs[0])

2426
page_content='atclNo: 2538700418
atclNm: 서울숲아이파크리버포레1차
region: 성수1가제2동
rletTpNm: 아파트
tradTpNm: 월세
flrInfo: 23/33
prc: 60000
rentPrc: 140
spc1: 81
spc2: 59.94
direction: 남서향
atclCfmYmd: 25.07.21.
lat: 37.551266
lng: 127.043521
atclFetrDesc: 지상철 소음없는 조용한동 선점 기회
bildNm: 104동
rltrNm: 청담워니공인중개사사무소' metadata={'source': './test_estate.csv', 'row': 0}


In [3]:
# 단계 2 : 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 2426


In [4]:
# 단계 3 : 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4 : DB 생성(Create DB) 및 저장
# 벡터 스토어를 생성합니다.
# !pip install faiss-cpu
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [5]:
# 단계 5 : 검색기 (Retriever) 생성

# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("월세 60이하, 6평 이상, 그리고 한양대학교와의 거리를 고려해서 월세 방을 5곳 추천해줘." \
"추천한 이유와 추천된 곳의 매물 이름, 월세, 보증금, 중개하는 부동산 이름을 같이 알려줘.")

[Document(id='b0ade20f-df4e-46b0-ac8e-7d78bac69f94', metadata={'source': './test_estate.csv', 'row': 769}, page_content='atclNo: 2539384948\natclNm: 일반원룸\nregion: 행당제2동\nrletTpNm: 원룸\ntradTpNm: 월세\nflrInfo: 5/5\nprc: 3000\nrentPrc: 45\nspc1: -\nspc2: 16.52\ndirection: 북서향\natclCfmYmd: 25.07.21.\nlat: 37.560466\nlng: 127.043284\natclFetrDesc: 월세저렴 풀옵션원룸 한양대인근 조용한주택가 보증금조절가능\nbildNm: \nrltrNm: 더드림부동산'),
 Document(id='85b1fbbb-3da6-45e6-a2a0-588c584ea55a', metadata={'source': './test_estate.csv', 'row': 1664}, page_content='atclNo: 2534647685\natclNm: 도시형생활주택\nregion: 응봉동\nrletTpNm: 원룸\ntradTpNm: 월세\nflrInfo: 3/5\nprc: 3000\nrentPrc: 45\nspc1: -\nspc2: 14.08\ndirection: 북향\natclCfmYmd: 25.06.27.\nlat: 37.556521\nlng: 127.041602\natclFetrDesc: 한양대 정문 도보 1분, 구조 좋고 깨끗, 주택가 위치\nbildNm: \nrltrNm: 봄날공인중개사사무소'),
 Document(id='8299164f-dc46-4e48-af95-35d28b0cb823', metadata={'source': './test_estate.csv', 'row': 2144}, page_content='atclNo: 2534175414\natclNm: 도시형생활주택\nregion: 성수1가제1동\nrletTpNm: 

In [6]:
# 단계 6 : 프롬프트 생성(Create Prompt)

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
    User the following peices of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Anser in Korean.
    retrieved context includes information of Deposit(column anme 'prc'), Monthly Rent(column name 'rentPrc'),
    Total Floor Area(in Korean, 공급면적. column name 'spc1'), Exclusive Use Area(in Korean, 전용면적. column name 'spc2'),
    Listing type(or Housing type. It includes 'apartment, studio aparmtent, officetel etc'. column name 'rletTpNm').
    District(column name 'region)'.
    And also include 'prc per spc1 and spc2 (column name 'prc/spc1' and 'prc/spc2)',
    'rentPrc per spc1 and spc2 (column name 'rentPrc/spc1' and 'rentPrc/spc2')'.
    All data come from Real Estat Listing Website. And I extract about Seongdong-Gu list, which is closed to Hanyang University, Korea.
    And all data is related to region District and Listing type.
    
    #Question:
    {question}

    #Context:
    {context}

    #Answer:
    """    
)

In [7]:
# 단계 7 : 언어모델(LLM) 생성

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [8]:
# 단계 8 : 체인(chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm    
    | StrOutputParser()
)

# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "월세 60이하, 6평 이상, 그리고 한양대학교와의 거리를 고려해서 월세 방을 5곳 추천해줘." \
"추천한 이유와 추천된 곳의 매물 이름, 월세, 보증금, 중개하는 부동산 이름을 같이 알려줘."
response = chain.invoke(question)
print(response)

1. **매물 이름: 일반원룸**
   - **월세:** 45만 원
   - **보증금:** 3000만 원
   - **중개하는 부동산 이름:** 더드림부동산
   - **추천 이유:** 한양대 인근에 위치하며, 조용한 주택가에 있어 생활하기 좋습니다. 보증금 조절이 가능하여 유연한 조건을 제공합니다.

2. **매물 이름: 도시형생활주택**
   - **월세:** 45만 원
   - **보증금:** 3000만 원
   - **중개하는 부동산 이름:** 봄날공인중개사사무소
   - **추천 이유:** 한양대 정문에서 도보 1분 거리로 매우 가깝고, 구조가 좋고 깨끗한 주택가에 위치해 있습니다.

위의 두 매물은 월세가 60만 원 이하이며, 한양대학교와의 거리가 가까워 추천드립니다. 나머지 매물은 월세 조건에 맞지 않거나 면적 조건에 부합하지 않아 제외하였습니다.


** CSV 병합 후 텍스트 변환 연습 코드 **

In [36]:
import pandas as pd
from pathlib import Path

folder = Path("./ex_data/")
dfs = []
for file in folder.glob("*.csv"):
    df = pd.read_csv(file)
    df['source'] = file.name  # 출처 기록
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

def row_to_text(row):
    return f"{row['region']} 지역의 {row['rletTpNm']}은 가격 {row['prc']}, 전세 {row['rentPrc']}, 면적 {row['spc1']} 평입니다. (출처: {row['source']})"

texts = df_all.apply(row_to_text, axis=1).tolist()

print('------text print-------')
print(texts)

------text print-------
['금호1가동 지역의 nan은 가격 15439.473684210529, 전세 122.23684210526316, 면적 73.91891891891892 평입니다. (출처: df_region_price.csv)', '금호2가동 지역의 nan은 가격 18457.894736842107, 전세 170.0, 면적 84.83333333333333 평입니다. (출처: df_region_price.csv)', '금호3가동 지역의 nan은 가격 18823.529411764703, 전세 206.843137254902, 면적 93.68627450980392 평입니다. (출처: df_region_price.csv)', '금호4가동 지역의 nan은 가격 7600.0, 전세 110.95555555555556, 면적 62.75 평입니다. (출처: df_region_price.csv)', '기타지역(1120012200) 지역의 nan은 가격 9110.21505376344, 전세 95.9462365591398, 면적 55.17486338797814 평입니다. (출처: df_region_price.csv)', '마장동 지역의 nan은 가격 2331.5789473684213, 전세 53.60526315789474, 면적 70.86111111111111 평입니다. (출처: df_region_price.csv)', '사근동 지역의 nan은 가격 6461.728395061728, 전세 72.23456790123457, 면적 45.65384615384615 평입니다. (출처: df_region_price.csv)', '성수1가제1동 지역의 nan은 가격 28906.25, 전세 276.85714285714283, 면적 110.07339449541284 평입니다. (출처: df_region_price.csv)', '성수1가제2동 지역의 nan은 가격 24392.88056206089, 전세 474.655737704918, 면적 83.57446808510639 평입니